In [47]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from collections import Counter
from statsmodels.sandbox.stats.multicomp import multipletests 

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', header = 0)
#data.columns = ['Placebo', 'Methylphenidate']

In [32]:
data_norm = data[data['Diagnosis'] == 'normal']
data_norm = data_norm.drop(columns=['Patient_id', 'Diagnosis'])

data_neop = data[data['Diagnosis'] == 'early neoplasia']
data_neop = data_neop.drop(columns=['Patient_id', 'Diagnosis'])

data_cancer = data[data['Diagnosis'] == 'cancer']
data_cancer = data_cancer.drop(columns=['Patient_id', 'Diagnosis'])

In [79]:
res1 = scipy.stats.ttest_ind(data_norm, data_neop, equal_var = False)[1]

In [80]:
Counter(res1 < 0.05)

Counter({False: 14173, True: 1575})

In [69]:
res2 = scipy.stats.ttest_ind(data_neop, data_cancer, equal_var = False)[1]

In [70]:
Counter(res2 < 0.05)

Counter({False: 12258, True: 3490})

In [95]:
reject, p_corrected, a1, a2 = multipletests(res1, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [96]:
Counter(p_corrected<0.025)

Counter({False: 15746, True: 2})

In [98]:
for counter, value in enumerate(p_corrected<0.025):
    if value == True:
        print(counter, value)

(7244, True)
(9820, True)


In [111]:
def fold_change(x,y):
    a = x.mean()
    b = y.mean()
    if b > a:
        return float(b)/a
    else:
        return -float(a)/b

In [112]:
data_norm.iloc[:,7244].mean()

14.983424243666663

In [114]:
print fold_change(data_norm.iloc[:,7244],data_neop.iloc[:,7244])
print fold_change(data_norm.iloc[:,9820],data_neop.iloc[:,9820])

1.50978548204
1.97486766564


In [119]:
ind_list = []
reject, p_corrected, a1, a2 = multipletests(res2, 
                                            alpha = 0.025, 
                                            method = 'holm') 
for counter, value in enumerate(p_corrected<0.025):
    if value == True:
        ind_list.append(counter)

In [123]:
abs_list = []
for i in np.array(ind_list):
    abs_list.append(abs(fold_change(data_neop.iloc[:,i],data_cancer.iloc[:,i])))

In [129]:
Counter(np.array(abs_list) > 1.5)

Counter({False: 2, True: 77})

In [136]:
ind_list = []
reject, p_corrected, a1, a2 = multipletests(res1, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 
for counter, value in enumerate(p_corrected<0.025):
    if value == True:
        ind_list.append(counter)

In [137]:
abs_list = []
for i in np.array(ind_list):
    abs_list.append(abs(fold_change(data_norm.iloc[:,i],data_neop.iloc[:,i])))

In [138]:
Counter(np.array(abs_list) > 1.5)

Counter({True: 4})

In [139]:
ind_list = []
reject, p_corrected, a1, a2 = multipletests(res2, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 
for counter, value in enumerate(p_corrected<0.025):
    if value == True:
        ind_list.append(counter)
abs_list = []
for i in np.array(ind_list):
    abs_list.append(abs(fold_change(data_neop.iloc[:,i],data_cancer.iloc[:,i])))
Counter(np.array(abs_list) > 1.5)

Counter({False: 308, True: 524})